In [68]:
"""
Mr.Kataei 8/4/2021
get_candle_api use in stream to get candles for analysis and strategies this method have limit for n last candles
you need for analysis.data collect form Bitfinex API where url can change for any API you want.
there is 3 type of dictionary for bitfinex symbols or CSvs we need to save
_get_all_candles is private method to use in generate_big_data where collect all data that can get from bitfinex
"""
import time
import pandas as pd
import requests
import enum


class Exchange(enum.Enum):
    binance = 'binance'
    bitfinex = 'bitfinex'


class Symbols(enum.Enum):
    BTCUSDT = 'bitcoin'
    ETHUSDT = 'ethereum'
    ADAUSDT = 'cardano'
    DOGEUSDT = 'doge'
    BCHUSDT = 'bitcoinCash'
    ETCUSDT = 'ethereumClassic'


symbols = {'BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOGEUSDT', 'BCHUSDT', 'ETCUSDT'}

symbols_bitfinix = {'BTCUSDT': 'tBTCUSD', 'ETHUSDT': 'tETHUSD', 'ADAUSDT': 'tADAUSD', 'DOGEUSDT': 'tDOGE:USD',
                    'BCHUSDT': 'tBCHN:USD', 'ETCUSDT': 'tETCUSD'}


def get_url_params_column(exchange: Exchange, timeframe: str, symbol: str, start_time: int = 115133520000):
    # 115133520000 is Wednesday, 21 December 2005 02:52:00
    if exchange.name == Exchange.binance.name:
        params = {'interval': timeframe, 'symbol': symbol, 'limit': 1000, 'startTime': start_time}
        url = 'https://api1.binance.com/api/v3/klines'
        columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'Qav', 'trade-number', 'TBbav',
                   'TBbqv', 'ignore']
    elif exchange.name == Exchange.bitfinex.name:
        symbol = symbols_bitfinix[symbol]
        params = {'limit': 10000, 'sort': 1, 'start': start_time}
        url = f'https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:{symbol}/hist'
        columns = ['date', 'open', 'close', 'high', 'low', 'volume']
    else:
        return
    return params, url, columns


def calculate_end_time(number: int = 4, unit: str = 'h'):
    unit = unit.lower()
    convert_time_to_ms = 0
    if unit == 'h':
        convert_time_to_ms = 3600 * 1000
    elif unit == 'm':
        convert_time_to_ms = 60 * 1000
    elif unit == 'd':
        convert_time_to_ms = 3600 * 1000 * 24
    elif unit == 'w':
         convert_time_to_ms = 3600 * 1000 * 24 * 7
    end_time = int(round(time.time() * 1000))
    return end_time - number * convert_time_to_ms


def get_all_candles_binance(exchange: Exchange, symbol: Symbols, number: int = 4, unit: str = 'h',
                            is_tehran: bool = True, save_csv: bool = True):
    """
    :param exchange: its enum from Exchange ( bitfinex , binance ,...)
    :param symbol: its enum from Symbols
    :param number: its int number number of day or hours or minutes
    :param unit: its string for which timeframe,  m for minutes , h for hours , d for day
    :param is_tehran: for casting to tehran time this parameter need to be true
    :param save_csv:  for save data to directory calls this function needs to be true
    :return: dataframe of ohcl
    """
    timeframe = str(number) + unit
    symbol = symbol.name
    params, url, columns = get_url_params_column(exchange=exchange, timeframe=timeframe, symbol=symbol)
    if params is None:
        return

    end_time = calculate_end_time(number=number, unit=unit)

    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=columns).astype(float)
        data = data[data.columns[:]]
        start_time = int(data.tail(1)['date'].values[0])
        while start_time < end_time:
            params, url, columns = get_url_params_column(exchange=exchange, timeframe=timeframe, symbol=symbol,
                                                         start_time=start_time)
            r = requests.get(url=url, params=params)
            next_data = r.json()
            next_data = pd.DataFrame(data=next_data, columns=columns).astype(float)
            next_data = next_data[data.columns[:]]
            data = pd.concat([data, next_data], axis=0)
            data = data.reset_index(drop=True)
            start_time = int(data.tail(1)['date'].values[0])
        if is_tehran:
            data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)).tz_localize(
                'UTC').tz_convert('Asia/Tehran')
        data = data.drop_duplicates(subset=['date'])
        if save_csv:
            data.to_csv(f'{symbol}-{timeframe}-{exchange.name}.csv', index=False)
        return data
    except Exception as e:
        print(f'something wrong on get data from {exchange.name}:\n', e)


def get_candle_bitfinex(symbol: str, timeframe: str, limit: int):
    symbol = symbols_bitfinix[symbol]
    params = {'limit': limit, 'sort': -1}
    url = f'https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:{symbol}/hist'
    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=['date', 'open', 'close', 'high', 'low', 'volume'])
        data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)
                                     ).tz_localize('UTC').tz_convert('Asia/Tehran')
        data = data.iloc[::-1]
        data = data.reset_index(drop=True)
        return data
    except Exception as e:
        print('something wrong on get data from bitfinex:\n', e)


def get_candle_binance(symbol: str, limit: int, number: int = 4, unit: str = 'h'):
    timeframe = str(number) + unit
    end_time = calculate_end_time(number=number, unit=unit)
    params = {'interval': timeframe, 'symbol': symbol, 'limit': limit, 'endTime': end_time}
    url = 'https://api1.binance.com/api/v3/klines'
    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                                'Qav', 'trade-number', 'TBbav', 'TBbqv', 'ignore']).astype(float)
        data = data[data.columns[0:6]]
        data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)).tz_localize(
            'UTC').tz_convert(
            'Asia/Tehran')
        return True, data
    except Exception as e:
        result = 'something wrong getting data from binance:\n' + str(e)
        return False, result


# usage
# ex = Exchange.binance
# sym = Symbols.BTCUSDT
# get_all_candles_binance(exchange=ex, symbol=sym, number=15, unit='m')

In [60]:
import pandas_ta as ta
import pandas as pd
import sys
import os

# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path+"/Libraries")
# # import sys
# # sys.path.append('~/Libraries/')

# from Libraries.data_collector import *

exchange = Exchange.bitfinex
symbol = Symbols.ETHUSDT


data = get_all_candles_binance(exchange=exchange, symbol=symbol, number=1, unit='h', save_csv=False)

In [61]:
import numpy as np
np.shape(data)
data.to_csv('../Datasets/bitfinex_ETHUSDT1h.csv')

In [58]:
data.sort_values(by=['date'],ascending=[True],inplace=True)
data.head()

,date,open,close,high,low,volume
0,2016-03-09 19:30:00+03:30,10.297,11.097,11.197,10.097,0.060000
1,2016-03-09 23:30:00+03:30,11.195,11.197,11.197,10.660,41.109523
2,2016-03-10 19:30:00+03:30,11.072,11.750,11.850,11.072,1.030000
3,2016-03-11 11:30:00+03:30,11.850,11.850,11.850,11.850,0.030000
4,2016-03-11 19:30:00+03:30,11.800,11.800,11.800,11.800,0.714898


In [69]:

exchange = Exchange.bitfinex
symbol = Symbols.ETHUSDT
# array = [[1,'w'],[1,'d'],[12,'h'],[6,'h'],[4,'h'],[3,'h'],[1,'h'],[30,'m'],[15,'m'],[5,'m'],[1,'m']]
array = [[1,'W'],[1,'D']]
for number,unit in array:
    get_all_candles_binance(exchange=exchange, symbol=symbol, number= number, unit=unit, save_csv=True)
